In [0]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www-eu.apache.org/dist/spark/spark-2.3.3/spark-2.3.3-bin-hadoop2.7.tgz
!tar xf spark-2.3.3-bin-hadoop2.7.tgz
!pip install -q findspark

In [0]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.3.3-bin-hadoop2.7"

In [0]:
import findspark
findspark.init()
from pyspark.sql import SparkSession

spark = SparkSession.builder.master("local[*]").getOrCreate()

In [0]:
import re;
from pyspark import SparkConf, SparkContext, HiveContext;
from pyspark.mllib.recommendation import ALS, MatrixFactorizationModel, Rating
from heapq import nlargest;

In [0]:
from pyspark import SparkConf
from pyspark import SparkContext
from pyspark import HiveContext
sc = SparkContext.getOrCreate(SparkConf().setMaster("local[*]"))

In [0]:
sqlContext = HiveContext(sc);

In [0]:
movie = sc.textFile("C:/Personal/Spring 2019/Analytics for Big Data/ABD_Assignments/ABD_Assignment 5/ml-100k/u.data");
movie1 = movie.map(lambda l: l.split('\t'));
movie_data = movie1.map(lambda a:(a[0],a[1],a[2]));

In [0]:
rank = 11;
numIterations = 10;
reg_factor = 0.05

model = ALS.train(movie_data, rank, seed=555, numIterations, reg_factor);
movie_test = movie_data.map(lambda p: (p[0], p[1]));
movie_pred = model.predictAll(movie_test).map(lambda r: (r[0], r[1], r[2]));
movie_prediction = model_pred.sortBy(lambda a:(a[0],a[2]),ascending=False);

In [0]:
movie_pred_a = model_prediction.groupBy(lambda x: x[0]).flatMap(lambda g: nlargest(10, g[1], key=lambda x: x[2]));
movie_pred_b = movie_pred_a.map(lambda a:(a[0],a[1]));
movie_pred_c = movie_pred_b.groupByKey().mapValues(list);

In [0]:
def csv(data):
return '\t'.join(str(d) for d in data);

In [0]:
movie_output = movie_pred_c.map(csv);

In [0]:
rating_output = movie_output.map(lambda a:re.sub('\[','',a));
rating_output_final = rating_output.map(lambda a:re.sub(']','',a));

In [0]:
rating_output_final.saveAsTextFile("hw5_final_output");